<a href="https://colab.research.google.com/github/Ahtesham-Ibne-Mostafa/Natural_Language_Processing/blob/main/Mimic_iv_BHC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
discharge = pd.read_csv(r"D:\Datasets\mimic-iv-ext-bhc-labeled-clinical-notes-dataset-for-hospital-course-summarization-1.1.0\discharge.csv")


In [4]:
patients= pd.read_csv(r"D:\Datasets\mimic-iv-ext-bhc-labeled-clinical-notes-dataset-for-hospital-course-summarization-1.1.0\patients.csv")

In [5]:
bhc=pd.read_csv(r"D:\Datasets\mimic-iv-ext-bhc-labeled-clinical-notes-dataset-for-hospital-course-summarization-1.1.0\mimic-iv-bhc.csv")

In [6]:
discharge

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...
...,...,...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,\nName: ___ Unit No: ___...
331789,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,\nName: ___ Unit No: ___...
331790,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...
331791,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...


In [7]:
patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN
299708,19999829,F,28,2186,2008 - 2010,NaN
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,NaN


In [8]:
patients.isnull().sum()

subject_id                0
gender                    0
anchor_age                0
anchor_year               0
anchor_year_group         0
dod                  270636
dtype: int64

In [9]:
bhc

,note_id,input,target,input_tokens,target_tokens
0,10000032-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1946,231
1,10000032-DS-22,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Percoce...,"___ with HIV on HAART, HCV cirrhosis with asci...",2183,810
2,10000117-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> omepraz...,Ms. ___ is a ___ with history of GERD who pres...,1060,172
3,10000117-DS-22,<SEX> F <SERVICE> ORTHOPAEDICS <ALLERGIES> ome...,The patient presented to the emergency departm...,1195,330
4,10000248-DS-10,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,Mr. ___ is a ___ with history of mild FVIII de...,1961,230
...,...,...,...,...,...
270028,19999784-DS-9,<SEX> M <SERVICE> NEUROSURGERY <ALLERGIES> No ...,Mr. ___ is a ___ without significant medical h...,1434,1133
270029,19999828-DS-6,<SEX> F <SERVICE> SURGERY <ALLERGIES> Lamictal...,Ms. ___ is a ___ year old female with a histor...,2517,442
270030,19999840-DS-20,<SEX> M <SERVICE> NEUROLOGY <ALLERGIES> Patien...,___ year old man hx of epilepsy and left tempo...,2775,233
270031,19999840-DS-21,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Patient...,Mr. ___ was admitted with status epilepticus. ...,1761,531


In [27]:
input_text=bhc['input'][4]
features=[]
a=''
flag=False
for i in input_text:
    if i == '<':
        flag=True
    if flag== True:
        a+=i
    if i == '>':
        features.append(a)
        a=''
        flag=False
print(len(features))
print(features)

20
['<SEX>', '<SERVICE>', '<ALLERGIES>', '<ATTENDING>', '<CHIEF COMPLAINT>', '<MAJOR SURGICAL OR INVASIVE PROCEDURE>', '<HISTORY OF PRESENT ILLNESS>', '', '<PAST MEDICAL HISTORY>', '<SOCIAL HISTORY>', '<FAMILY HISTORY>', '<PHYSICAL EXAM>', '<PERTINENT RESULTS>', '<1 WBC-1 Bacteri-FEW Yeast-NONE Epi-0 ___ 07: 50PM URINE Mucous-RARE IMAGING/STUDIES: ___ CT A/P Acute hemorrhage along right posterior flank musculature and probably layering over it, only partly imaged and hard to distinguish musculature from hemorrhage. No active extravasation seen. Probable old hematoma along posterior left flank. <MEDICATIONS ON ADMISSION>', '<DISCHARGE MEDICATIONS>', '<DISCHARGE DISPOSITION>', '<DISCHARGE DIAGNOSIS>', '<DISCHARGE CONDITION>', '<FOLLOWUP INSTRUCTIONS>', '<DISCHARGE INSTRUCTIONS>']


In [19]:
input_text=bhc['input'][0]
# Find all values between <>
matches = re.findall(r'<(.*?)>', input_text)

# Print the results
print(len(matches))

19


In [26]:
input_text=bhc['input'][1]
# Find all values between <>
matches = re.findall(r'<(.*?)>', input_text)

# Print the results
print(matches)

['SEX', 'SERVICE', 'ALLERGIES', 'ATTENDING', 'CHIEF COMPLAINT', 'MAJOR SURGICAL OR INVASIVE PROCEDURE', 'HISTORY OF PRESENT ILLNESS', 'PAST MEDICAL HISTORY', 'SOCIAL HISTORY', 'FAMILY HISTORY', 'PHYSICAL EXAM', 'PERTINENT RESULTS', '1 per 1000X FIELD): POLYMORPHONUCLEAR LEUKOCYTES. NO MICROORGANISMS SEEN. This is a concentrated smear made by cytospin method, please refer to hematology for a quantitative white blood cell count.. FLUID CULTURE (Final ___: NO GROWTH. ANAEROBIC CULTURE (Preliminary): NO GROWTH. ___ 7: 00 pm PERITONEAL FLUID PERITONEAL FLUID. GRAM STAIN (Final ___: 1+ (<1 per 1000X FIELD): POLYMORPHONUCLEAR LEUKOCYTES. NO MICROORGANISMS SEEN. This is a concentrated smear made by cytospin method, please refer to hematology for a quantitative white blood cell count.. FLUID CULTURE (Final ___: NO GROWTH. ANAEROBIC CULTURE (Preliminary): NO GROWTH. Diagnositc Para: ___ 07: 00PM ASCITES ___ ___ ___ 07: 00PM ASCITES ___ IMAGING: ___ CXR- No acute cardiopulmonary abnormality. ___ 

In [27]:
bhc['input'][0]

'<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Known Allergies / Adverse Drug Reactions <ATTENDING> ___ <CHIEF COMPLAINT> Worsening ABD distension and pain <MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis <HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone ___ weeks ago, because she feels like "they don\'t do anything" and that she "doesn\'t want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing

In [28]:
bhc['input'][1]

"<SEX> F <SERVICE> MEDICINE <ALLERGIES> Percocet <ATTENDING> ___. <CHIEF COMPLAINT> abdominal fullness and discomfort <MAJOR SURGICAL OR INVASIVE PROCEDURE> ___ diagnostic paracentesis ___ therapeutic paracentesis <HISTORY OF PRESENT ILLNESS> ___ with HIV on HAART, COPD, HCV cirrhosis complicated by ascites and HE admitted with abdominal distention and pain. She was admitted to ___ for the same symptoms recently and had 3L fluid removed (no SBP) three days ago and felt better. Since discharge, her abdomen has become increasingly distended with pain. This feels similar to prior episodes of ascites. Her diuretics were recently decreased on ___ due to worsening hyponatremia 128 and hyperkalemia 5.1. Patient states she has been compliant with her HIV and diuretic medications but never filled out the lactulose prescription. She states she has had ___ BMs daily at home. She has had some visual hallucinations and forgetfulness. Her appetite has been poor. In the ED, initial vitals were 98.9 8

In [21]:
bhc['target'][0]

'___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week and confusion. # Ascites - p/w worsening abd distension and discomfort for last week. likely ___ portal HTN given underlying liver disease, though no ascitic fluid available on night of admission. No signs of heart failure noted on exam. This was ___ to med non-compliance and lack of diet restriction. SBP negative diuretics: > Furosemide 40 mg PO DAILY > Spironolactone 50 mg PO DAILY, chosen over the usual 100mg dose d/t K+ of 4.5. CXR was wnl, UA negative, Urine culture blood culture negative. Pt was losing excess fluid appropriately with stable lytes on the above regimen. Pt was scheduled with current PCP for ___ check upon discharge. Pt was scheduled for new PCP with Dr. ___ at ___ and follow up in Liver clinic to schedule outpatient screening EGD and ___. '